1) Импортируем `pymorphy2` (эта чудная библиотека умеет много чего, в том числе ставить слова в начальную форму и определять часть речи, но работает медленно). `Counter`, `pandas` и регулярки нам также пригодятся

In [25]:
import pymorphy2
import pandas as pd
from collections import Counter
import re

2) Открываем текст и делим его на слова с помощью регулярных выражений

In [26]:
words=open('The three ages2.txt').read()
words=re.split(r'\W+', words)

3) Создаем словарь, в котором ключами будут слова из текста, а значениями – количество их повторений. В цикле приводим словаа в начальную форму, избавляемся от случайных пустых строк и служебных частей речи 

In [27]:
vocab=[]
for i in words:
    m=pymorphy2.MorphAnalyzer().parse(i)[0]
    if (i and m.tag.POS not in ['PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO']):
        vocab.append(m.normal_form)
vocab=dict(Counter(vocab))

4) Создаем `DataFrame` из .csv с частотным словарем русского языка, складываем значения для повторяющихся слов (в этом словаре записаны, например, несколько "а" как разные части речи)

In [28]:
freq=pd.read_csv('freqrnc2011.csv', sep='\t', index_col='Lemma')
freq=freq.groupby(level=0).sum()

5) Создаем отдельный список для имен, названий, кличек и т.д., которых нет в частотном словаре (сюда попадают и слова типа "россия", которые в словаре записаны как "Россия", а также слова с ё, увы). Для слов из нашего текста, которые в словаре есть, частоту приводим к общей с .csv единице измерения (слов на миллион) и делим на значение из частотного словаря, убирая из топа слова, которые просто чаще всего в русском языке встречаются

In [29]:
names=[]
for j in vocab:
    if j in freq.index:
        vocab[j]*=1000000/len(words)
        vocab[j]=round(vocab[j]/freq.loc[j, 'Freq(ipm)'], 2)
    else:
        names.append([j, vocab[j]])

6) Сортируем наш словарь (покажем первые 30 и первые 20 списка `names`)

In [30]:
for i in sorted(list(vocab.items()), key=lambda x: x[1], reverse=True)[:30]:
    print(i)
print()
for j in names[:20]:
    print(j)

('клипер', 14880.95)
('траверз', 4960.32)
('швартовый', 4960.32)
('цокот', 3306.88)
('аксельбант', 2834.47)
('непристойно', 2834.47)
('вассал', 2480.16)
('тормашки', 2480.16)
('настораживаться', 2480.16)
('ливрея', 2204.59)
('эполет', 2204.59)
('вульгарно', 1984.13)
('подбочениться', 1984.13)
('раут', 1984.13)
('кавалькада', 1803.75)
('принесение', 1653.44)
('отрубать', 1653.44)
('стяг', 1526.25)
('гарцевать', 1526.25)
('каравай', 1417.23)
('андреевский', 1417.23)
('платан', 1417.23)
('мандарин', 1280.08)
('искренно', 1240.08)
('неусыпный', 1240.08)
('обсадить', 1240.08)
('резня', 1102.29)
('поднебесный', 1044.28)
('повиновение', 1044.28)
('пуп', 793.65)

['китай', 4]
['цзунлиямынь', 1]
['4', 1]
['ятсный', 1]
['1964', 1]
['250', 1]
['251', 1]
['богдыхан', 1]
['франция', 1]
['россия', 3]
['пекин', 1]
['канонерка', 1]
['чайковский', 4]
['кульджинский', 1]
['уйгур', 1]
['дунганин', 1]
['семиречье', 1]
['цзунлиямыня', 1]
['вели', 1]
['англия', 1]
